## This code is used for scraping images along with their locations from Flickr 

In [16]:
import flickrapi
import urllib.request
import csv
import time
import os.path
from datetime import datetime
from PIL import Image
import requests

# insert your own key and secret
apiKey = 'b0c368bb882f13acb59583b4d9aefca9'
secretApiKey = 'f98331af23ed43da'
flickr = flickrapi.FlickrAPI(apiKey, secretApiKey, format='parsed-json')

csvColumns = ['id','owner','secret','server','farm','title','ispublic','isfriend',
            'isfamily','description','dateupload','datetaken','datetakengranularity',
            'datetakenunknown','ownername','latitude','longitude','accuracy',
            'context','place_id','woeid','geo_is_family','geo_is_friend',
            'geo_is_contact','geo_is_public','url_c','height_c','width_c']

def initiateFile(filename):
    csvFileName = filename + ".csv"
    with open(csvFileName, 'w') as csvFile:
        writer = csv.DictWriter(csvFile, fieldnames=csvColumns)
        writer.writeheader()
    csvFile.close()
    
def getPhotos(keyword, pageNr, minDate):
    response = flickr.photos.search(
        text=keyword,
        tag_mode='any',
        tags=keyword,
        per_page=250,
        extras=["url_c, geo, description, date_upload, date_taken, owner_name"],
        sort='date-posted-asc',
        has_geo="1",
        page=pageNr,
        min_upload_date=minDate,
        format='parsed-json'
    )
    return response

def getMinDate(filename):
    csvFileName = filename + ".csv"
    with open(csvFileName, 'r') as csvFile:
        reader = csv.DictReader(csvFile)
        try:
            r = list(reader)[-1]
            return(int(r['dateupload']))
        except IndexError:
            return 0

def downloadImage(photo, directory):
    if 'url_c' not in photo:
        return None
    
    url = photo['url_c']
    lat = photo['latitude']
    lon = photo['longitude']
    if url and lat and lon:
        filename = photo['id'] + ".jpg"
        filePath = os.path.join(directory, filename)
        urllib.request.urlretrieve(url, filePath)
        return filePath
    else:
        return None

def walkBatch(keyword, filename, directory):
    errors = 0

    csvFileName = filename + ".csv"

    if not os.path.exists(csvFileName):
        initiateFile(filename)
        minDate = 0
    else:
        minDate = getMinDate(filename)

    maxPages = 15
    pageNr = 1
    #total_images = 0
    #images_downloaded = 0

    while pageNr < maxPages and errors < 10:
        try:
            post = getPhotos(keyword, pageNr, minDate)
            photos = post['photos']['photo']
            
            if not photos:
                break

            with open(csvFileName, 'a') as csvFile:
                writer = csv.DictWriter(csvFile, fieldnames=csvColumns)
                for photo in photos:
                    lat = photo['latitude']
                    lon = photo['longitude']
                    if lat and lon:
                        writer.writerow(photo)
                        downloadImage(photo, directory)
                    
            csvFile.close()
                   
            date = datetime.fromtimestamp(int(list(photos)[-1]['dateupload']))
            
            print("page: " + str(pageNr) + ", photos: " + str(len(photos)))
            pageNr += 1
            

        except flickrapi.exceptions.FlickrError:
            print("Oops")
            errors += 1
        except requests.exceptions.ConnectionError:
            print("Oops")
            errors += 1


while True:
    walkBatch("Istanbul", "Istanbullocations_01", "images_01")
    time.sleep(10)


page: 1, photos: 250
page: 2, photos: 250
page: 3, photos: 250
page: 4, photos: 250
page: 5, photos: 250
page: 6, photos: 250


KeyboardInterrupt: 

#### The code is in never ending loop to scrape as many pages as possible. Manual Interruption is required to stop the code